# নমুনা ০২: OpenAI SDK ইন্টিগ্রেশন

এই নোটবুকটি OpenAI Python SDK-এর সাথে উন্নত ইন্টিগ্রেশন প্রদর্শন করে, যা Microsoft Foundry Local এবং Azure OpenAI উভয়ের জন্য স্ট্রিমিং রেসপন্স এবং সঠিক ত্রুটি পরিচালনা সমর্থন করে।

## সংক্ষিপ্ত বিবরণ

এই নমুনাটি প্রদর্শন করে:
- Foundry Local এবং Azure OpenAI-এর মধ্যে সহজে সুইচ করার ক্ষমতা
- স্ট্রিমিং চ্যাট কমপ্লিশনস ব্যবহারকারীর অভিজ্ঞতা উন্নত করার জন্য
- FoundryLocalManager SDK-এর সঠিক ব্যবহার
- শক্তিশালী ত্রুটি পরিচালনা এবং ব্যাকআপ ব্যবস্থা
- প্রোডাকশন-রেডি কোড প্যাটার্ন


## প্রয়োজনীয়তা

- **Foundry Local**: ইনস্টল করা এবং চালু (লোকাল ইনফারেন্সের জন্য)
- **Python**: 3.8 বা পরবর্তী সংস্করণ OpenAI SDK সহ
- **Azure OpenAI**: বৈধ এন্ডপয়েন্ট এবং API কী (ক্লাউড ইনফারেন্সের জন্য)

### নির্ভরতা ইনস্টল করুন


In [ ]:
# Install required packages
!pip install openai foundry-local-sdk

## লাইব্রেরি আমদানি এবং সেটআপ


In [ ]:
import os
import sys
from openai import OpenAI
import time
from typing import Tuple

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, manual configuration will be used")

## কনফিগারেশন অপশন

সঠিক পরিবেশ ভেরিয়েবল সেট করে Azure OpenAI (ক্লাউড) অথবা Foundry Local (অন-ডিভাইস) এর মধ্যে নির্বাচন করুন।


### অপশন ১: Azure OpenAI কনফিগারেশন

আপনার Azure OpenAI পরিচয়পত্র আনকমেন্ট করুন এবং সেট করুন:


In [ ]:
# Azure OpenAI Configuration
# Uncomment and set your actual values

# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key-here"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"  # e.g., "gpt-4"

print("Azure OpenAI configuration ready (if credentials are set)")

### অপশন ২: ফাউন্ড্রি লোকাল কনফিগারেশন

লোকাল ইনফারেন্সের জন্য ডিফল্ট সেটিংস:


In [ ]:
# Foundry Local Configuration (default)
FOUNDRY_MODEL = "phi-4-mini"  # Change to your preferred model
FOUNDRY_BASE_URL = "http://localhost:51211"
FOUNDRY_API_KEY = ""  # Usually empty for local

print(f"Foundry Local configuration ready with model: {FOUNDRY_MODEL}")

## ক্লায়েন্ট ফ্যাক্টরি ফাংশনসমূহ

এই ফাংশনগুলো আপনার কনফিগারেশন অনুযায়ী সঠিক OpenAI ক্লায়েন্ট তৈরি করে:


In [ ]:
def create_azure_client() -> Tuple[OpenAI, str]:
    """Create Azure OpenAI client."""
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if not azure_endpoint or not azure_api_key:
        raise ValueError("Azure OpenAI endpoint and API key are required")
    
    model = os.environ.get("MODEL", "your-deployment-name")
    client = OpenAI(
        base_url=f"{azure_endpoint}/openai",
        api_key=azure_api_key,
        default_query={"api-version": azure_api_version},
    )
    
    print(f"🌐 Azure OpenAI client created with model: {model}")
    return client, model


def create_foundry_client() -> Tuple[OpenAI, str]:
    """Create Foundry Local client with SDK management."""
    alias = FOUNDRY_MODEL
    
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            print(f"🔄 Initializing Foundry Local with model: {alias}...")
            manager = FoundryLocalManager(alias)
            model_info = manager.get_model_info(alias)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            
            print(f"✅ Foundry Local SDK initialized")
            print(f"   Endpoint: {manager.endpoint}")
            print(f"   Model: {model_info.id}")
            return client, model_info.id
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{FOUNDRY_BASE_URL}/v1",
        api_key=FOUNDRY_API_KEY
    )
    
    print(f"🔧 Manual Foundry Local configuration")
    print(f"   Endpoint: {FOUNDRY_BASE_URL}/v1")
    print(f"   Model: {alias}")
    return client, alias

## ক্লায়েন্ট আরম্ভ করুন

এটি স্বয়ংক্রিয়ভাবে নির্ধারণ করে Azure OpenAI বা Foundry Local ব্যবহার করা হবে কিনা:


In [ ]:
def initialize_client() -> Tuple[OpenAI, str, str]:
    """Initialize the appropriate OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    
    if azure_endpoint and azure_api_key:
        print("🌐 Azure OpenAI configuration detected")
        try:
            client, model = create_azure_client()
            return client, model, "azure"
        except Exception as e:
            print(f"❌ Azure OpenAI initialization failed: {e}")
            print("🔄 Falling back to Foundry Local...")
    
    # Use Foundry Local
    print("🏠 Using Foundry Local configuration")
    try:
        client, model = create_foundry_client()
        return client, model, "foundry"
    except Exception as e:
        print(f"❌ Foundry Local initialization failed: {e}")
        raise

# Initialize the client
print("Initializing OpenAI client...")
print("=" * 50)
client, model, provider = initialize_client()
print("=" * 50)
print(f"✅ Initialization complete! Using {provider} with model: {model}")

## সাধারণ চ্যাট সম্পন্ন করা

একটি সহজ চ্যাট সম্পন্ন পরীক্ষা করুন:


In [ ]:
def simple_chat(prompt: str, max_tokens: int = 150) -> str:
    """Send a simple chat message and get response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test basic chat
test_prompt = "Say hello from the SDK quickstart and explain what you are in one sentence."

print(f"👤 User: {test_prompt}")
print("\n🤖 Assistant:")
response = simple_chat(test_prompt)
print(response)

## স্ট্রিমিং চ্যাট সম্পূর্ণতা

উন্নত ব্যবহারকারীর অভিজ্ঞতার জন্য স্ট্রিমিং প্রতিক্রিয়া প্রদর্শন করুন:


In [ ]:
def streaming_chat(prompt: str, max_tokens: int = 300) -> str:
    """Send a chat message with streaming response."""
    try:
        print("🤖 Assistant (streaming):")
        
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        error_msg = f"Error: {e}"
        print(error_msg)
        return error_msg

# Test streaming chat
streaming_prompt = "Explain the key benefits of using Microsoft Foundry Local for AI development. Include aspects like privacy, performance, and cost."

print(f"👤 User: {streaming_prompt}\n")
streaming_response = streaming_chat(streaming_prompt)

## বহু-পর্বের কথোপকথন

কথোপকথনের প্রেক্ষাপট বজায় রাখার উদাহরণ দিন:


In [ ]:
class ConversationManager:
    """Manages multi-turn conversations with context."""
    
    def __init__(self, system_prompt: str = None):
        self.messages = []
        if system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})
    
    def send_message(self, user_message: str, max_tokens: int = 200) -> str:
        """Send a message and get response while maintaining context."""
        # Add user message to conversation
        self.messages.append({"role": "user", "content": user_message})
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=self.messages,
                max_tokens=max_tokens
            )
            
            assistant_message = response.choices[0].message.content
            
            # Add assistant response to conversation
            self.messages.append({"role": "assistant", "content": assistant_message})
            
            return assistant_message
        except Exception as e:
            return f"Error: {e}"
    
    def get_conversation_length(self) -> int:
        """Get the number of messages in the conversation."""
        return len(self.messages)

# Create conversation manager with system prompt
system_prompt = "You are a helpful AI assistant specialized in explaining AI and machine learning concepts. Be concise but informative."
conversation = ConversationManager(system_prompt)

# Multi-turn conversation example
conversation_turns = [
    "What is the difference between AI inference on-device vs in the cloud?",
    "Which approach is better for privacy?",
    "What about performance and latency considerations?"
]

for i, turn in enumerate(conversation_turns, 1):
    print(f"\n{'='*60}")
    print(f"Turn {i}")
    print(f"{'='*60}")
    print(f"👤 User: {turn}")
    
    response = conversation.send_message(turn)
    print(f"\n🤖 Assistant: {response}")

print(f"\n📊 Conversation summary: {conversation.get_conversation_length()} messages total")

## পারফরম্যান্স তুলনা

বিভিন্ন পরিস্থিতির জন্য প্রতিক্রিয়া সময় তুলনা করুন:


In [ ]:
def benchmark_response_time(prompt: str, iterations: int = 3) -> dict:
    """Benchmark response time for a given prompt."""
    times = []
    responses = []
    
    for i in range(iterations):
        start_time = time.time()
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=50  # Keep responses short for timing
            )
            
            end_time = time.time()
            response_time = end_time - start_time
            
            times.append(response_time)
            responses.append(response.choices[0].message.content)
            
        except Exception as e:
            print(f"Error in iteration {i+1}: {e}")
    
    if times:
        avg_time = sum(times) / len(times)
        min_time = min(times)
        max_time = max(times)
        
        return {
            "average_time": avg_time,
            "min_time": min_time,
            "max_time": max_time,
            "all_times": times,
            "sample_response": responses[0] if responses else None
        }
    
    return {"error": "No successful responses"}

# Benchmark different types of prompts
benchmark_prompts = [
    "What is AI?",
    "Explain machine learning in simple terms.",
    "List 3 benefits of edge computing."
]

print(f"⏱️  Performance Benchmark ({provider} - {model})")
print("=" * 60)

for prompt in benchmark_prompts:
    print(f"\n📝 Prompt: '{prompt}'")
    results = benchmark_response_time(prompt)
    
    if "error" not in results:
        print(f"   ⏰ Average time: {results['average_time']:.2f}s")
        print(f"   ⚡ Fastest: {results['min_time']:.2f}s")
        print(f"   🐌 Slowest: {results['max_time']:.2f}s")
        print(f"   📄 Sample response: {results['sample_response'][:100]}...")
    else:
        print(f"   ❌ {results['error']}")

## উন্নত কনফিগারেশন এবং ত্রুটি পরিচালনা

বিভিন্ন প্যারামিটার এবং ত্রুটি পরিস্থিতি পরীক্ষা করুন:


In [ ]:
def test_different_parameters():
    """Test chat completions with different parameters."""
    prompt = "Write a creative short story about AI."
    
    # Test different temperature values
    temperatures = [0.1, 0.5, 0.9]
    
    for temp in temperatures:
        print(f"\n🌡️ Temperature: {temp}")
        print("-" * 30)
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,
                temperature=temp
            )
            
            print(f"Response: {response.choices[0].message.content[:150]}...")
            
        except Exception as e:
            print(f"Error with temperature {temp}: {e}")

test_different_parameters()

## সার্ভিস স্বাস্থ্য পরীক্ষা

সার্ভিসের স্বাস্থ্য এবং সক্ষমতার সম্পূর্ণ পরীক্ষা:


In [ ]:
def comprehensive_health_check():
    """Perform comprehensive health check of the service."""
    print("🏥 Comprehensive Health Check")
    print("=" * 50)
    
    # 1. Check model listing
    try:
        models_response = client.models.list()
        available_models = [m.id for m in models_response.data]
        print(f"✅ Model listing: SUCCESS")
        print(f"   📋 Available models: {available_models}")
        
        if model in available_models:
            print(f"   ✅ Current model '{model}' is available")
        else:
            print(f"   ⚠️ Current model '{model}' not found in available models")
    except Exception as e:
        print(f"❌ Model listing: FAILED - {e}")
    
    # 2. Test basic completion
    try:
        test_response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Say 'Health check successful'"}],
            max_tokens=10
        )
        print(f"✅ Basic completion: SUCCESS")
        print(f"   💬 Response: {test_response.choices[0].message.content}")
    except Exception as e:
        print(f"❌ Basic completion: FAILED - {e}")
    
    # 3. Test streaming
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Count to 3"}],
            max_tokens=20,
            stream=True
        )
        
        stream_content = ""
        chunk_count = 0
        for chunk in stream:
            if chunk.choices[0].delta.content:
                stream_content += chunk.choices[0].delta.content
                chunk_count += 1
        
        print(f"✅ Streaming: SUCCESS")
        print(f"   📦 Chunks received: {chunk_count}")
        print(f"   💬 Streamed content: {stream_content.strip()}")
    except Exception as e:
        print(f"❌ Streaming: FAILED - {e}")
    
    # 4. Provider-specific information
    print(f"\n📊 Configuration Summary:")
    print(f"   🏢 Provider: {provider}")
    print(f"   🤖 Model: {model}")
    if provider == "foundry":
        print(f"   🏠 Foundry SDK Available: {FOUNDRY_SDK_AVAILABLE}")
        print(f"   🔗 Base URL: {FOUNDRY_BASE_URL}")
    elif provider == "azure":
        print(f"   🌐 Azure Endpoint: {os.environ.get('AZURE_OPENAI_ENDPOINT', 'Not set')}")
        print(f"   🔑 API Version: {os.environ.get('AZURE_OPENAI_API_VERSION', 'Not set')}")

comprehensive_health_check()

## ইন্টারঅ্যাকটিভ টেস্টিং

এই সেলটি ব্যবহার করে আপনার নিজস্ব প্রম্পটগুলি ইন্টারঅ্যাকটিভভাবে পরীক্ষা করুন:


In [ ]:
# Interactive testing - modify the prompt below
custom_prompt = "Explain the concept of 'edge AI' and why it's becoming important."
use_streaming = True  # Set to False for regular completion

print(f"👤 Custom Prompt: {custom_prompt}\n")

if use_streaming:
    custom_response = streaming_chat(custom_prompt, max_tokens=250)
else:
    custom_response = simple_chat(custom_prompt, max_tokens=250)
    print(f"🤖 Assistant: {custom_response}")

## সারসংক্ষেপ এবং পরবর্তী পদক্ষেপ

এই নোটবুকটি উন্নত OpenAI SDK ইন্টিগ্রেশন প্রদর্শন করেছে:

### ✅ কাভার করা মূল বৈশিষ্ট্যসমূহ

1. **মাল্টি-প্রোভাইডার সাপোর্ট**: Azure OpenAI এবং Foundry Local-এর মধ্যে সহজে সুইচিং
2. **স্ট্রিমিং রেসপন্স**: উন্নত UX-এর জন্য রিয়েল-টাইম টোকেন জেনারেশন
3. **কনভারসেশন ম্যানেজমেন্ট**: কনটেক্সট সহ মাল্টি-টার্ন কথোপকথন
4. **পারফরম্যান্স বেঞ্চমার্কিং**: রেসপন্স টাইম পরিমাপ এবং বিশ্লেষণ
5. **সম্পূর্ণ স্বাস্থ্য পরীক্ষা**: সার্ভিস যাচাই এবং ডায়াগনস্টিকস
6. **এরর হ্যান্ডলিং**: শক্তিশালী এরর হ্যান্ডলিং এবং ফলব্যাক মেকানিজম

### 🏆 Foundry Local বনাম Azure OpenAI

| দিক | Foundry Local | Azure OpenAI |
|-----|---------------|--------------|
| **গোপনীয়তা** | ✅ ডেটা লোকাল থাকে | ⚠️ ডেটা ক্লাউডে পাঠানো হয় |
| **লেটেন্সি** | ✅ কম (লোকাল ইনফারেন্স) | ⚠️ বেশি (নেটওয়ার্ক নির্ভরতা) |
| **খরচ** | ✅ বিনামূল্যে (হার্ডওয়্যার পরবর্তী) | 💰 প্রতি টোকেনের জন্য খরচ |
| **অফলাইন** | ✅ অফলাইনে কাজ করে | ❌ ইন্টারনেট প্রয়োজন |
| **মডেলের বৈচিত্র্য** | ⚠️ সীমিত নির্বাচন | ✅ সম্পূর্ণ মডেল অ্যাক্সেস |
| **স্কেলিং** | ⚠️ হার্ডওয়্যার নির্ভর | ✅ সীমাহীন স্কেলিং |

### 🚀 পরবর্তী পদক্ষেপ

- **Sample 04**: Chainlit চ্যাট অ্যাপ্লিকেশন তৈরি করা
- **Sample 05**: মাল্টি-এজেন্ট অর্কেস্ট্রেশন সিস্টেম
- **Sample 06**: ইন্টেলিজেন্ট মডেল রাউটিং
- **প্রোডাকশন ডিপ্লয়মেন্ট**: স্কেলিং এবং মনিটরিং বিবেচনা

### 💡 সেরা অনুশীলন

1. **প্রোভাইডারদের মধ্যে ফলব্যাক মেকানিজম** সর্বদা বাস্তবায়ন করুন
2. **দীর্ঘ রেসপন্সের জন্য স্ট্রিমিং ব্যবহার করুন** পারফরম্যান্স উন্নত করতে
3. **প্রোডাকশন অ্যাপ্লিকেশনের জন্য সঠিক এরর হ্যান্ডলিং** বাস্তবায়ন করুন
4. **বিভিন্ন প্রোভাইডারের জন্য রেসপন্স টাইম এবং খরচ মনিটর করুন**
5. **আপনার নির্দিষ্ট প্রয়োজন অনুযায়ী সঠিক প্রোভাইডার নির্বাচন করুন**
